In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
%%shell
git clone https://github.com/utkarsh512/CreateDebate-Scraper.git

In [ ]:
%%shell
pip install bs4
pip install glom

In [ ]:
# Uncomment to start scraping
#
# %%shell
# cd CreateDebate-Scraper/src/nested/
# python scrape.py --page_count 104 --tag Politics --data_dir /content/gdrive/MyDrive/DL/CreateDebate/Politics

In [ ]:
%cd CreateDebate-Scraper/src/nested/

In [ ]:
reader_addr = '/content/gdrive/MyDrive/DL/CreateDebate/Politics/threads.log'

from thread import Thread, Comment
import pickle
from collections import OrderedDict
from wordcloud import WordCloud, STOPWORDS
from graphviz import Digraph

# Analysis of Dataset

In [ ]:
threadCount = 0
commentCount = 0

reader = open(reader_addr, 'rb')

threads = []

e = Thread()

try:
    while True:
        e = pickle.load(reader)
        threads.append(e)
except:
    reader.close()

In [ ]:
len(threads)

In [ ]:
maxCommentCount = 0

for x in threads:
    commentCount += len(x.comments)
    maxCommentCount = max(maxCommentCount, len(x.comments))

In [ ]:
print(commentCount, maxCommentCount)

In [ ]:
dis = [0] * (maxCommentCount + 1)
for x in threads:
    dis[len(x.comments)] += 1

In [ ]:
X = []
for i in range(maxCommentCount + 1):
    X.append(i)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(dpi=100)
plt.plot(X, dis)
plt.xlabel('# arguments')
plt.ylabel('# debates')
plt.title('Distribution of arguments in threads')
plt.show()

In [ ]:
v = OrderedDict()
v['0'] = dis[0]
v['1'] = dis[1]
v['2'] = dis[2]
v['3-5'] = dis[3] + dis[4] + dis[5]
v['6-10'] = 0
for i in range(6, 11):
    v['6-10'] += dis[i]
v['11-20'] = 0
for i in range(11, 21):
    v['11-20'] += dis[i]
v['>20'] = 0
for i in range(21, len(dis)):
    v['>20'] += dis[i]

In [ ]:
x = ['0', '1', '2', '3-5', '6-10', '11-20', '>20']
y = []
for xx in x:
    y.append(v[xx])
plt.bar(x, y)
plt.xlabel('Number of arguments per post')
plt.ylabel('Number of posts')
plt.savefig('distribution_of_number_of_arguments_per_post_create_debate.eps', format='eps', dpi=1200)

In [ ]:
# Printing any random thread

import random

idx = random.randint(0, len(threads) - 1)
print(threads[idx])

In [ ]:
# Extracting distribution of comments by their level

maxDepth = 0

def dfs(Map, cid='root', depth=0):
    global maxDepth
    maxDepth = max(maxDepth, depth)
    if cid == 'root':
        for key in Map[cid].keys():
            dfs(Map[cid], key, depth + 1)
        return

    for key in Map[cid].keys():
        dfs(Map[cid], key, depth + 1)

In [ ]:
for e in threads:
    if 'root' in e.metaL.keys():
        dfs(e.metaL)
    if 'root' in e.metaR.keys():
        dfs(e.metaR)

In [ ]:
print(maxDepth)

In [ ]:
cnt = [0] * (maxDepth + 1)

def dfs(Map, cid='root', depth=0, cnt=None):
    if depth != 0:
        cnt[depth] += 1
    if cid == 'root':
        for key in Map[cid].keys():
            dfs(Map[cid], key, depth + 1, cnt)
        return
    for key in Map[cid].keys():
        dfs(Map[cid], key, depth + 1, cnt)

In [ ]:
for e in threads:
    if 'root' in e.metaL.keys():
        dfs(e.metaL, cnt=cnt)
    if 'root' in e.metaR.keys():
        dfs(e.metaR, cnt=cnt)

In [ ]:
x = []
for i in range(maxDepth + 1):
    x.append(i)

In [ ]:
plt.figure(dpi = 100)
plt.plot(x[1:], cnt[1:])
plt.xlabel('Level')
plt.ylabel('Number of comments')
plt.title('Distribution of Comments (Level-wise)')
plt.show()

In [ ]:
v = dict()
v['1'] = cnt[1]
v['2'] = cnt[2]
v['3-5'] = cnt[3] + cnt[4] + cnt[5]
v['6-10'] = 0
for i in range(6, 11):
    v['6-10'] += cnt[i]
v['>10'] = 0
for i in range(11, len(cnt)):
    v['>10'] += cnt[i]

In [ ]:

x = ['1', '2', '3-5', '6-10', '>10']
y = []
for xx in x:
    y.append(v[xx])
plt.bar(x, y)
plt.xlabel('Level')
plt.ylabel('Number of arguments')
plt.savefig('distribution_of_number_of_arguments_on_each_level_create_debate.eps', format='eps', dpi=1200)

In [ ]:
# Extracting the length of comments in CreateDebate (both word-wise and character-wise)

maxLength = 0

for e in threads:
    for x in e.comments.keys():
        cmnt = e.comments[x]
        cur = len(cmnt.body.split())
        maxLength = max(maxLength, cur)

print(maxLength)

In [ ]:
cnt = [0 for x in range(maxLength + 1)]
for e in threads:
    for x in e.comments.keys():
        cmnt = e.comments[x]
        cnt[len(cmnt.body.split())] += 1

x = [i for i in range(maxLength + 1)]

plt.figure(dpi = 100)
plt.plot(x[1:], cnt[1:])
plt.xlabel('Level')
plt.ylabel('Number of comments')
plt.title('Distribution of Comments (Level-wise)')
plt.show()

In [ ]:
v = dict()
v['0-50'] = 0
for i in range(51):
    v['0-50'] += cnt[i]
v['51-100'] = 0
for i in range(51, 101):
    v['51-100'] += cnt[i]
v['101-200'] = 0
for i in range(101, 201):
    v['101-200'] += cnt[i]
v['201-500'] = 0
for i in range(201, 501):
    v['201-500'] += cnt[i]
v['501-1000'] = 0
for i in range(501, 1001):
    v['501-1000'] += cnt[i]
v['>1000'] = 0
for i in range(1001, maxLength + 1):
    v['>1000'] += cnt[i]

In [ ]:
# Extracting how often the authors comment

athr = dict()

text = ""

for e in threads:
    for x in e.comments.keys():
        cmnt = e.comments[x]
        cur_athr = cmnt.author
        text = text + " " + cur_athr
        try:
            athr[cur_athr] += 1
        except:
            athr[cur_athr] = 1

In [ ]:
len(athr)

In [ ]:
def genWordCloud(text, max_words=100):
    wordcloud = WordCloud(background_color='white', max_words=max_words, width=800, height=400, relative_scaling=1.0, stopwords=STOPWORDS).generate(text)
    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    plt.imshow(wordcloud)
    plt.show()

In [ ]:
genWordCloud(text)

In [ ]:
s = []
for x in athr.keys():
    s.append((athr[x], x))

In [ ]:
s = sorted(s, reverse=True)

In [ ]:
print(s[0])

In [ ]:
cnt = [0] * (9000)
for x in athr.keys():
    cnt[athr[x]] += 1

In [ ]:
x = []
for i in range(9000):
    x.append(i)

In [ ]:
plt.figure(dpi = 100)
plt.plot(x[1:50], cnt[1:50])
plt.xlabel('Comment count')
plt.ylabel('Number of authors')
plt.title('Distribution of Authors')
plt.show()

In [ ]:
v = dict()
v['1'] = cnt[1]
v['2'] = cnt[2]
v['3-5'] = cnt[3] + cnt[4] + cnt[5]
v['6-10'] = 0
for i in range(6, 11):
    v['6-10'] += cnt[i]
v['11-20'] = 0
for i in range(11, 21):
    v['11-20'] += cnt[i]
v['>20'] = 0
for i in range(21, len(cnt)):
    v['>20'] += cnt[i]

In [ ]:

x = ['1', '2', '3-5', '6-10', '11-20', '>20']
y = []
for xx in x:
    y.append(v[xx])
plt.bar(x, y)
plt.xlabel('Number of arguments')
plt.ylabel('Number of authors')
plt.savefig('distribution_of_authors_wrt_comment_frequency_create_debate.eps', format='eps', dpi=1200)

In [ ]:
athr = dict()
for e in threads:
    if 'root' in e.metaL.keys():
        for key in e.metaL['root'].keys():
            cmnt = e.comments[key]
            cur_athr = cmnt.author
            try:
                athr[cur_athr] += 1
            except:
                athr[cur_athr] = 1
    if 'root' in e.metaR.keys():
        for key in e.metaR['root'].keys():
            cmnt = e.comments[key]
            cur_athr = cmnt.author
            try:
                athr[cur_athr] += 1
            except:
                athr[cur_athr] = 1

In [ ]:
len(athr)

In [ ]:
s = []
for x in athr.keys():
    s.append((athr[x], x))
s = sorted(s, reverse=True)
for i in range(10):
    x, y = s[i]
    print(f'# L1: {x} \t Author: {y}')

In [ ]:
cnt = [0] * 1500
for x in athr:
    cnt[athr[x]] += 1
x = []
for i in range(1500):
    x.append(i)

In [ ]:
plt.figure(dpi = 100)
plt.plot(x[1:50], cnt[1:50])
plt.xlabel('Comment count')
plt.ylabel('Number of authors')
plt.title('Distribution of Authors')
plt.show()

In [ ]:
v = dict()
v['1'] = cnt[1]
v['2'] = cnt[2]
v['3-5'] = cnt[3] + cnt[4] + cnt[5]
v['6-10'] = 0
for i in range(6, 11):
    v['6-10'] += cnt[i]
v['11-20'] = 0
for i in range(11, 21):
    v['11-20'] += cnt[i]
v['>20'] = 0
for i in range(21, len(cnt)):
    v['>20'] += cnt[i]

In [ ]:

x = ['1', '2', '3-5', '6-10', '11-20', '>20']
y = []
for xx in x:
    y.append(v[xx])
plt.bar(x, y)
plt.xlabel('Number of level-1 arguments')
plt.ylabel('Number of authors')
plt.savefig('distribution_of_authors_wrt_level_1_comment_frequency_create_debate.eps', format='eps', dpi=1200)

In [ ]:
# Storing the authors who posted more than 200 L1 comments

L1_athr = dict()
for x in athr:
    if athr[x] > 200:
        L1_athr[x] = True

In [ ]:
# Popularity of author (by number of direct replies received)

athr = dict()

def dfs(Map, cmntMap, athr, cid='root'):
    if cid == 'root':
        for key in Map[cid].keys():
            dfs(Map[cid], cmntMap, athr, key)
        return
    cur_author = cmntMap[cid].author

    try:
        athr[cur_author] += len(Map[cid].keys())
    except:
        athr[cur_author] = len(Map[cid].keys())

    for key in Map[cid].keys():
        dfs(Map[cid], cmntMap, athr, key)

In [ ]:
for e in threads:
    if 'root' in e.metaL.keys():
        dfs(e.metaL, e.comments, athr)
    if 'root' in e.metaR.keys():
        dfs(e.metaR, e.comments, athr) 

In [ ]:
len(athr)

In [ ]:
s = []
for x in athr:
    s.append((athr[x], x))

s = sorted(s, reverse=True)

In [ ]:
for i in range(10):
    x, y = s[i]
    print(f'# Direct Replies: {x} \t Author: {y}')

In [ ]:
cnt = [0] * (6000)
for x in athr:
    cnt[athr[x]] += 1

In [ ]:
x = []
for i in range(6000):
    x.append(i)

In [ ]:
plt.figure(dpi = 100)
plt.plot(x[1:50], cnt[1:50])
plt.xlabel('# direct replies received')
plt.ylabel('# authors')
plt.title('Distribution of direct replies')
plt.show()

In [ ]:
v = dict()
v['1'] = cnt[1]
v['2'] = cnt[2]
v['3-5'] = cnt[3] + cnt[4] + cnt[5]
v['6-10'] = 0
for i in range(6, 11):
    v['6-10'] += cnt[i]
v['11-20'] = 0
for i in range(11, 21):
    v['11-20'] += cnt[i]
v['>20'] = 0
for i in range(21, len(cnt)):
    v['>20'] += cnt[i]

In [ ]:

x = ['1', '2', '3-5', '6-10', '11-20', '>20']
y = []
for xx in x:
    y.append(v[xx])
plt.bar(x, y)
plt.xlabel('Number of direct replies')
plt.ylabel('Number of authors')
plt.savefig('distribution_of_authors_wrt_direct_reply_frequency_create_debate.eps', format='eps', dpi=1200)

In [ ]:
# Storing the authors who posted more than 20 L1 comments

A = []
for x in athr:
    if athr[x] > 100:
        try:
            z = L1_athr[x]
            A.append(x)
        except KeyError:
            pass

In [ ]:
len(A)

In [ ]:
athr_id = dict()
which = [""] * len(A)
for i in range(len(A)):
    athr_id[A[i]] = i
    which[i] = A[i]
print(which)

In [ ]:
G = [[0 for i in range(len(A))] for j in range(len(A))]
H = [[0 for i in range(len(A))] for j in range(len(A))]

In [ ]:
def dfs(Map, cmntMap, cid='root'):
    if cid == 'root':
        for key in Map[cid].keys():
            dfs(Map[cid], cmntMap, key)
        return

    cur_author = cmntMap[cid].author
    cur_pol = cmntMap[cid].polarity
    
    if cur_author in athr_id and cur_pol != 'Not Available':
        cur_author_id = athr_id[cur_author]
        for key in Map[cid].keys():
            nxt_author = cmntMap[key].author
            nxt_pol = cmntMap[key].polarity
            if nxt_author in athr_id and nxt_pol != 'Not Available':
                nxt_author_id = athr_id[nxt_author]
                if cur_pol == nxt_pol:
                    G[nxt_author_id][cur_author_id] += 1
                else:
                    H[nxt_author_id][cur_author_id] += 1

    for key in Map[cid].keys():
        dfs(Map[cid], cmntMap, key)

In [ ]:
for e in threads:
    if 'root' in e.metaL:
        dfs(e.metaL, e.comments)
    if 'root' in e.metaR:
        dfs(e.metaR, e.comments)

In [ ]:
G_max = 0
H_max = 0

for i in range(len(A)):
    for j in range(len(A)):
        G_max = max(G_max, G[i][j])
        H_max = max(H_max, H[i][j])

In [ ]:
print(G_max, H_max)

In [ ]:
# Visualizing Support Graph

u = Digraph('unix', filename='unix.gv', node_attr={'color': 'lightblue2', 'style': 'filled'}, format='png')

for i in range(len(A)):
    for j in range(len(A)):
        node1 = which[i]
        node2 = which[j]
        if H[i][j] != 0:
            u.edge(node1, node2, label=str(H[i][j]))

In [ ]:
u.render(view=True)

In [ ]:
import numpy as np 

fig = plt.figure()
fig.dpi= 100
ax = fig.add_subplot(111)
cax = ax.matshow(H, interpolation='nearest')
fig.colorbar(cax)
ax.set_xticks(np.arange(len(A)))
ax.set_yticks(np.arange(len(A)))
ax.set_xticklabels(which)
ax.set_yticklabels(which)
plt.setp(ax.get_xticklabels(), rotation=-90)
plt.show()

In [ ]:
def Peek(author, max_words=200):
    text = ""
    for e in threads:
        for x in e.comments:
            cmnt = e.comments[x]
            if cmnt.author == author:
                text = text + ' ' + cmnt.body
    genWordCloud(text, max_words)

In [ ]:
Peek('outlaw60', 400)

# Community Detection

In [ ]:
authorMapL1 = {} # Contains author name who wrote atleast one L1 comment and thier index (in building the graphs)
authorCountL1 = 0

In [ ]:
for e in threads:
    if 'root' in e.metaL.keys():
        for key in e.metaL['root'].keys():
            cmnt = e.comments[key]
            curAuthor = cmnt.author
            if curAuthor not in authorMapL1:
                authorMapL1[curAuthor] = authorCountL1
                authorCountL1 += 1
    if 'root' in e.metaR.keys():
        for key in e.metaR['root'].keys():
            cmnt = e.comments[key]
            curAuthor = cmnt.author
            if curAuthor not in authorMapL1:
                authorMapL1[curAuthor] = authorCountL1
                authorCountL1 += 1

In [ ]:
G = [[0 for j in range(authorCountL1)] for i in range(authorCountL1)] # Support matrix
H = [[0 for j in range(authorCountL1)] for i in range(authorCountL1)] # Dispute matrix

In [ ]:
# Calculating support matrix and dispute matrix

athr_id = authorMapL1

def dfs(Map, cmntMap, cid='root'):
    if cid == 'root':
        for key in Map[cid].keys():
            dfs(Map[cid], cmntMap, key)
        return

    cur_author = cmntMap[cid].author
    cur_pol = cmntMap[cid].polarity
    
    if cur_author in athr_id and cur_pol != 'Not Available':
        cur_author_id = athr_id[cur_author]
        for key in Map[cid].keys():
            nxt_author = cmntMap[key].author
            nxt_pol = cmntMap[key].polarity
            if nxt_author in athr_id and nxt_pol != 'Not Available':
                nxt_author_id = athr_id[nxt_author]
                if cur_pol == nxt_pol:
                    G[nxt_author_id][cur_author_id] += 1
                else:
                    H[nxt_author_id][cur_author_id] += 1

    for key in Map[cid].keys():
        dfs(Map[cid], cmntMap, key)

for e in threads:
    if 'root' in e.metaL:
        dfs(e.metaL, e.comments)
    if 'root' in e.metaR:
        dfs(e.metaR, e.comments)

In [ ]:
G_max = 0
H_max = 0

for i in range(authorCountL1):
    for j in range(authorCountL1):
        G_max = max(G_max, G[i][j])
        H_max = max(H_max, H[i][j])

print(G_max, H_max)

In [ ]:
import numpy as np 

fig = plt.figure()
fig.dpi= 100
ax = fig.add_subplot(111)
cax = ax.matshow(G, interpolation='nearest')
fig.colorbar(cax)
plt.show()

In [ ]:
which = ["lol"] * (authorCountL1)
for key in authorMapL1.keys():
    which[authorMapL1[key]] = key

In [ ]:
import networkx as nx

In [ ]:
G_ = nx.DiGraph()
for i in range(authorCountL1):
    for j in range(authorCountL1):
        if G[i][j] != 0:
            G_.add_weighted_edges_from([(i, j, G[i][j])])

In [ ]:
H_ = nx.DiGraph()
for i in range(authorCountL1):
    for j in range(authorCountL1):
        if H[i][j] != 0:
            H_.add_weighted_edges_from([(i, j, H[i][j])])

In [ ]:
support_graph_reciprocity = nx.algorithms.reciprocity(G_)
dispute_graph_reciprocity = nx.algorithms.reciprocity(H_)

In [ ]:
print(support_graph_reciprocity, dispute_graph_reciprocity)